### Imports and set-up
---
> Uncomment as needed

In [ ]:
import numpy as np
import pandas as pd

# Training DataFrames:

trn_traits = pd.read_csv('training_data/trait_data.csv')
trn_metadata = pd.read_csv('training_data/meta_data.csv')
# trn_env_cov = pd.read_csv('training_data/env_cov.csv')
# trn_soil = pd.read_csv('training_data/soil_data.csv')
# trn_weather_season = pd.read_csv('training_data/weather_season.csv')
# trn_weather_year = pd.read_csv('training_data/weather_year.csv')


# Testing DataFrames:

# tst_template = pd.read_csv('testing_data/template.csv')
# tst_metadata = pd.read_csv('testing_data/meta_data.csv')
# tst_env_cov = pd.read_csv('testing_data/env_cov.csv')
# tst_soil = pd.read_csv('testing_data/soil_data.csv')
# tst_weather_season = pd.read_csv('testing_data/weather_season.csv')
# tst_weather_year = pd.read_csv('testing_data/weather_year.csv')

Helper functions:

In [142]:
R = 6371000 # Earth Radius (m)
def haversine_distance(lat1, lat2, lon1, lon2):
    lat1, lat2, lon1, lon2 = map(np.radians, [lat1, lat2, lon1, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return c*R

#### Defining Useful Column Subsets
---

This section is for meta_data

In [ ]:
# Condensing 4 positions into 1 (center location)
lat_cols = [c for c in trn_metadata.columns if c.startswith('Latitude')]
lon_cols = [c for c in trn_metadata.columns if c.startswith('Longitude')]

trn_metadata.loc[:, 'Latitude'] = trn_metadata[lat_cols].mean(axis=1)
trn_metadata.loc[:, 'Longitude'] = trn_metadata[lon_cols].mean(axis=1)

In [ ]:
metadata_cols_important = [
    'Year', 'Env', 'Experiment_Code', 
    'City', 'Farm', 'Field',
    'Latitude', 'Longitude'
]

metadata_cols_interesting = [
    'Year', 'Env', 'Experiment_Code', 
    'City', 'Farm', 'Field',
    'Pre-plant_tillage_method(s)', 'In-season_tillage_method(s)',
    'Previous_Crop', 'Irrigated',
    'Latitude', 'Longitude'
]

metadata_cols_minimal = ['Env', 'Latitude', 'Longitude']

trn_metadata = trn_metadata[metadata_cols_important]
trn_metadata.head()

,Year,Env,Experiment_Code,City,Farm,Field,Latitude,Longitude
0,2014,DEH1_2014,DEH1,Georgetown,Elbert N. & Ann V. Carvel Research & Education...,27AB,NaN,NaN
1,2014,GAH1_2014,GAH1,Tifton,USDA - Bellflower experimental farm,18,NaN,NaN
2,2014,IAH1a_2014,IAH1,Ames,Worle,NaN,NaN,NaN
3,2014,IAH1b_2014,IAH1,Ames,Worle,NaN,NaN,NaN
4,2014,IAH1c_2014,IAH1,Ames,Worle,NaN,NaN,NaN


This section is for trait_data:

In [145]:
# TODO: define useful column subsets for trait_data

#### Addressing Missing Locations
---

This works by first trying to match each Env with a missing location to the most similar Env whose location can be copied.

The remaining values are just selected by hand.

In [ ]:
# First, self-join meta_data on Experiment_Code and City.
# This makes a table where each Env has a number of rows that share at least Experiment_Code and City.
merged = trn_metadata.merge(
    trn_metadata,  # Join with itself
    on=['Experiment_Code', 'City'], 
    suffixes=('', '_other')
)

# Filter out self matches, and narrow down to rows that are missing their location, but the comparable Env still has a location.
merged = merged[(merged['Year'] != merged['Year_other']) & merged['Latitude'].isna() & merged['Latitude_other'].notna()]

# Now, most of the Envs with a missing location will have multiple comparable Envs
# Assign each of those other Envs a match_priority value, which determines how closely they match
# Two Envs that share a 'Field' will have priority value 3, sharing a 'Farm' will get 2, and only sharing 'City' will get 1.
merged['match_priority'] = (
    (merged['Field'] == merged['Field_other']).astype(int) +
    (merged['Farm'] == merged['Farm_other']).astype(int) +
    1
)

merged = merged.sort_values(by=['Env', 'match_priority'], ascending=[True, False]) # Sort to put the highest priority match for each Env first.
best_matches = merged.groupby('Env').first().reset_index() # Get one row per Env again, but only keep the highest priority match
best_matches = best_matches[['Env', 'Latitude_other', 'Longitude_other']] # Narrow down to just important columns

# I have manually determined which Envs were not able to find a match, and found a suitable location:
curated_locations = [
    ['ILH1_2017', 40.060724, 88.233881],                    # Choosing MF-500, copying from ILH1_2016
    ['INH1_2017', 40.478760, 86.989820],                    # Choosing Purdue Acre 54 North, copying from INH1_2016
    ['NEH2_2019', 40.86073680542686, -96.6139217242634],    # Choosing a larger field near the Havelock Research Farm in Lincoln, NE
    ['TXH2_2014', 34.19196292257452, -101.96587766472442],  # Chose a random center pivot field after searching 'Halfway, TX'
    ['TXH2_2015', 33.683219771598594, -101.8228099023082],  # | For these, I am just going with a field next to the
    ['TXH2_2016', 33.683219771598594, -101.8228099023082],  # | Texas A&M AgriLife Research & Extension Center at Lubbock
    ['TXH2_2017', 33.683219771598594, -101.8228099023082],  # | 
    ['TXH2_2018', 33.683219771598594, -101.8228099023082],  # | It should be accurate for the ones listed as Lubbock, and close enough for the others. 
    ['TXH4_2019', 33.683219771598594, -101.8228099023082]   # /
]
curated_locations_df = pd.DataFrame(curated_locations, columns=['Env', 'Latitude_other', 'Longitude_other'])
best_matches = pd.concat([best_matches, curated_locations_df], ignore_index=True) # Concatenate my curated locations to the matches.

# Finally, I align my new locations with the meta_data df, and fill in the missing values using my new values.
aligned_matches = trn_metadata.merge(best_matches, on='Env', how='left')
trn_metadata['Latitude'] = trn_metadata['Latitude'].fillna(aligned_matches['Latitude_other'])
trn_metadata['Longitude'] = trn_metadata['Longitude'].fillna(aligned_matches['Longitude_other'])

print('Missing Locations:')
trn_metadata[trn_metadata['Latitude'].isna()] # print the misisng locations to verify they are all taken care of 

Missing Locations:


,Year,Env,Experiment_Code,City,Farm,Field,Latitude,Longitude
22,2014,TXH2_2014,TXH2,Halfway,Halfway,pivot,NaN,NaN
48,2015,TXH2_2015,TXH2,NaN,NaN,NaN,NaN,NaN
75,2016,TXH2_2016,TXH2,NaN,NaN,NaN,NaN,NaN
88,2017,ILH1_2017,ILH1,NaN,NaN,NaN,NaN,NaN
89,2017,INH1_2017,INH1,NaN,NaN,NaN,NaN,NaN
105,2017,TXH2_2017,TXH2,NaN,NaN,NaN,NaN,NaN
134,2018,TXH2_2018,TXH2,Lubbock,NaN,NaN,NaN,NaN
152,2019,NEH2_2019,NEH2,Lincoln,NaN,NaN,NaN,NaN
162,2019,TXH4_2019,TXH4,Lubbock,NaN,NaN,NaN,NaN


#### Envirotyping Input
---

Input Variables:
| Variable | Description | Collection Method |
| --- | --- | --- |
| Site | User site Identifier | Use 'Env' column |
| Planting | Plating Date (mm/dd/yyyy) | use 'Date_Planted' column |
| Latitude | Latitude of trial | use meta_data['Latitude'] |
| Longitude | Longitude of trial | use meta_data['Longitude'] |
| Crop | soybean or maize | 'maize' |
| Genetics | Soybean: maturity group (0-6, by 1), Maize: RM (80-130, by 5) | ??? |

In [ ]:
et_trait_cols = ['Env', 'Field_Location', 'Year', 'Date_Planted']
# et_genetic_cols = ['Hybrid', 'Hybrid_orig_name'] # Unfortunately, none of the materials provided seem to list maturities for the hybrids
et_timing_cols = ['Pollen_DAP_days', 'Silk_DAP_days', 'Date_Harvested']

et_input = trn_traits[et_trait_cols + et_timing_cols]
et_input = et_input.merge(trn_metadata[metadata_cols_minimal], on='Env')

# Convert Date columns to_datetime
et_input['Date_Planted'] = pd.to_datetime(et_input['Date_Planted'], format='%m/%d/%y')
if 'Date_Harvested' in et_input: et_input['Date_Harvested'] = pd.to_datetime(et_input['Date_Harvested'], format='%m/%d/%y')

et_input['Harvest_DAP_days'] = (et_input['Date_Harvested'] - et_input['Date_Planted']).dt.days
et_input

,Env,Field_Location,Year,Date_Planted,Pollen_DAP_days,Silk_DAP_days,Date_Harvested,Latitude,Longitude,Harvest_DAP_days
0,DEH1_2014,DEH1,2014,2014-05-05,63.0,67.0,2014-09-29,38.629357,-75.465693,147.0
1,DEH1_2014,DEH1,2014,2014-05-05,61.0,63.0,2014-09-29,38.629357,-75.465693,147.0
2,DEH1_2014,DEH1,2014,2014-05-05,63.0,65.0,2014-09-29,38.629357,-75.465693,147.0
3,DEH1_2014,DEH1,2014,2014-05-05,61.0,63.0,2014-09-29,38.629357,-75.465693,147.0
4,DEH1_2014,DEH1,2014,2014-05-05,63.0,65.0,2014-09-29,38.629357,-75.465693,147.0
...,...,...,...,...,...,...,...,...,...,...
173955,WIH3_2023,WIH3,2023,2023-04-26,81.0,82.0,2023-11-14,44.115645,-89.544009,202.0
173956,WIH3_2023,WIH3,2023,2023-04-26,70.0,70.0,2023-11-14,44.115645,-89.544009,202.0
173957,WIH3_2023,WIH3,2023,2023-04-26,78.0,80.0,2023-11-14,44.115645,-89.544009,202.0
173958,WIH3_2023,WIH3,2023,2023-04-26,84.0,100.0,2023-11-14,44.115645,-89.544009,202.0


# Testing
---

Everything below here is notes and experiments in progress.

In [128]:
# The purpose of this cell is to try to fill in missing value in `cols_to_fill`
# It kinda works, but there are still 9 Pollen, and 10 Silk days that aren't filled since there is no data for IAH1/TXH4 to use.
agg_func = lambda x: x.mode().min() # Aggregate columns by finding the most common value for that group (and break ties with min)

cols_to_fill = ['Pollen_DAP_days', 'Silk_DAP_days', 'Date_Harvested']
per_env = et_input.groupby('Env', as_index=False).agg(agg_func)

# What this does is get a row for each location (e.g. ILH1) and has the (smallest) mode for that value,
# Then any gaps in the data that was grouped by 'Env' are filled by the data that was grouped by the larger 'Field_Location'
test = per_env.merge(et_input.groupby('Field_Location').agg({c: agg_func for c in cols_to_fill}), on='Field_Location', how='left')
for col in cols_to_fill:
    test[col] = test[f'{col}_x'].fillna(test[f'{col}_y'])
    test = test.drop(columns=[f'{col}_x', f'{col}_y'])

test

,Env,Field_Location,Year,Date_Planted,Latitude,Longitude,Harvest_DAP_days,Pollen_DAP_days,Silk_DAP_days,Date_Harvested
0,ARH1_2016,ARH1,2016,2016-04-07,34.729520,-90.760356,145.0,67.0,69.0,2016-08-30
1,ARH1_2017,ARH1,2017,2017-04-17,34.727252,-90.760326,147.0,65.0,71.0,2017-09-11
2,ARH1_2018,ARH1,2018,2018-04-25,34.729679,-90.760345,129.0,58.0,69.0,2018-09-01
3,ARH2_2016,ARH2,2016,2016-04-23,35.838614,-90.665302,200.0,61.0,62.0,2016-11-09
4,ARH2_2017,ARH2,2017,2017-04-25,35.674181,-90.075270,144.0,62.0,64.0,2017-09-16
...,...,...,...,...,...,...,...,...,...,...
267,WIH2_2023,WIH2,2023,2023-05-11,43.304243,-89.384146,200.0,78.0,79.0,2023-11-27
268,WIH3_2020,WIH3,2020,2020-05-21,44.115703,-89.544913,171.0,62.0,63.0,2020-11-08
269,WIH3_2021,WIH3,2021,2021-04-29,44.114305,-89.544225,159.0,76.0,78.0,2021-10-05
270,WIH3_2022,WIH3,2022,2022-05-16,44.119050,-89.556345,169.0,71.0,70.0,2022-11-01


In [ ]:
# Find the best values for these columns to fill in gaps by copying over valuse from the most similar trial(s)
per_env = et_input.groupby(['Field_Location', 'Env']).agg({
    'Date_Planted': lambda x: x.mode().min(), 
    'Pollen_DAP_days': lambda x: x.mode().min(),
    'Silk_DAP_days': lambda x: x.mode().min(),
    'Date_Harvested': lambda x: x.mode().min()
}).reset_index()


# Backup values to fill in the ones that are still missing after grouping by Env
per_floc = trn_traits.groupby('Field_Location').agg({
    'Pollen_DAP_days': lambda x: x.mode().min(),
    'Silk_DAP_days': lambda x: x.mode().min(),
    'Date_Harvested': lambda x: x.mode().min()
})

test = per_env.merge(per_floc, on='Field_Location', how='left')

merge_cols = ['Pollen_DAP_days', 'Silk_DAP_days', 'Date_Harvested']
for col in merge_cols:
    test[col] = test[f'{col}_x'].fillna(test[f'{col}_y'])
    test = test.drop(columns=[f'{col}_x', f'{col}_y'])

# per_env
# per_floc
# et_input['Date_Planted'] = et_input['Date_Planted'].fillna(et_input.merge(plantings, on='Env', how='left')['Date_Planted_y'])
# test

Pollen_DAP_days       0
Silk_DAP_days         0
Date_Harvested     7572
dtype: int64

In [68]:
# Consider trying to fill in planting/harvest dates by looking at the other crops in the same plot/experiment/year
planting_dates = et_input[['Env', 'Date_Planted']].drop_duplicates(subset='Env', keep='first').reset_index(drop=True)

aligned_planting_dates = et_input.merge(planting_dates, on='Env', how='left')

aligned_planting_dates

,Env,Year,Date_Planted_x,Pollen_DAP_days,Silk_DAP_days,Date_Harvested,Latitude,Longitude,Harvest_DAP_days,Date_Planted_y
0,DEH1_2014,2014,2014-05-05,63.0,67.0,2014-09-29,38.629357,-75.465693,147.0,2014-05-05
1,DEH1_2014,2014,2014-05-05,61.0,63.0,2014-09-29,38.629357,-75.465693,147.0,2014-05-05
2,DEH1_2014,2014,2014-05-05,63.0,65.0,2014-09-29,38.629357,-75.465693,147.0,2014-05-05
3,DEH1_2014,2014,2014-05-05,61.0,63.0,2014-09-29,38.629357,-75.465693,147.0,2014-05-05
4,DEH1_2014,2014,2014-05-05,63.0,65.0,2014-09-29,38.629357,-75.465693,147.0,2014-05-05
...,...,...,...,...,...,...,...,...,...,...
173955,WIH3_2023,2023,2023-04-26,81.0,82.0,2023-11-14,44.115645,-89.544009,202.0,2023-04-26
173956,WIH3_2023,2023,2023-04-26,70.0,70.0,2023-11-14,44.115645,-89.544009,202.0,2023-04-26
173957,WIH3_2023,2023,2023-04-26,78.0,80.0,2023-11-14,44.115645,-89.544009,202.0,2023-04-26
173958,WIH3_2023,2023,2023-04-26,84.0,100.0,2023-11-14,44.115645,-89.544009,202.0,2023-04-26


In [37]:
test = et_input.groupby(['Env', 'Date_Planted']).agg({
    'Year': 'first', 'Latitude': 'first', 'Longitude': 'first',
    'Pollen_DAP_days': ['mean', 'std'], 'Silk_DAP_days': ['mean', 'std'], 'Date_Harvested': [set, 'nunique'], 'Harvest_DAP_days': ['mean', 'std']
}).reset_index()
test

Env Date_Planted  Year   Latitude  Longitude Pollen_DAP_days  \
                            first      first      first            mean   
0    ARH1_2016   2016-04-07  2016  34.729520 -90.760356       69.004090   
1    ARH1_2017   2017-04-17  2017  34.727252 -90.760326       66.238208   
2    ARH1_2018   2018-04-25  2018  34.729679 -90.760345       60.249509   
3    ARH2_2016   2016-04-23  2016  35.838614 -90.665302       63.277433   
4    ARH2_2017   2017-04-25  2017  35.674181 -90.075270       63.654255   
..         ...          ...   ...        ...        ...             ...   
283  WIH2_2023   2023-05-11  2023  43.304243 -89.384146       78.613148   
284  WIH3_2020   2020-05-21  2020  44.115703 -89.544913       61.591216   
285  WIH3_2021   2021-04-29  2021  44.114305 -89.544225       77.511905   
286  WIH3_2022   2022-05-16  2022  44.119050 -89.556345       69.752595   
287  WIH3_2023   2023-04-26  2023  44.115645 -89.544009       80.892086   

              Silk_DAP_days                   Date_Harvested          \
          std          mean       std                    set nunique   
0    2.658882     70.580777  3.107718  {2016-08-30 00:00:00}       1   
1    3.328699     67.530660  3.108595  {2017-09-11 00:00:00}       1   
2    2.682442     65.487230  3.525823  {2018-09-01 00:00:00}       1   
3    2.907575     64.801242  3.187410  {2016-11-09 00:00:00}       1   
4    3.454486     65.316489  3.583420  {2017-09-16 00:00:00}       1   
..        ...           ...       ...                    ...     ...   
283  3.058423     79.927939  3.166456  {2023-11-27 00:00:00}       1   
284  4.950017     63.625000  5.060840  {2020-11-08 00:00:00}       1   
285  3.929731     79.077453  4.472813  {2021-10-05 00:00:00}       1   
286  3.429817     71.300518  3.477488  {2022-11-01 00:00:00}       1   
287  3.720778     83.621928  4.968916  {2023-11-14 00:00:00}       1   

    Harvest_DAP_days       
                mean  std  
0              145.0  0.0  
1              147.0  0.0  
2              129.0  0.0  
3              200.0  0.0  
4              144.0  0.0  
..               ...  ...  
283            200.0  0.0  
284            171.0  0.0  
285            159.0  0.0  
286            169.0  0.0  
287            202.0  0.0  

[288 rows x 13 columns]

In [8]:
et_input.to_csv('output_data/envirotype_input.csv')

In [ ]:
# Goal 1: Get a single dataframe that contains all of the most important information for the competition
metadata_cols_important = [
    'Env', 
    'Location_Code',
    'Year',
    'Latitude',
    'Longitude',
    'Hybrid',
    'Date_Planted',
    # Date_Harvested, Pollen_DAP_days, Silk_DAP_days, Harvest_DAP_days?
    # Check trait_data for some useful phenotype measurements
    # Yield
    # Decide whether we want any of the columns from metadata, like irrigation, methods, etc.
]

# Or, maybe make a few dfs? 
# I could have:
#  - minimal meta_data
#  - meta_data extra info
#  - minimal trait_data
#  - trait_data extra info
#  - 

In [ ]:
# Goal 2: Make a naive method of estimating yield by finding the year with the most similar conditons to 2024 and 
# copying over the yield values from that year for the given hybrids/locations